# IMPORTING REQUIRED LIBRARIES

In [1]:
from selenium import webdriver                          #For opening browser       
import pandas as pd                                     #For creating dataframe
from bs4 import BeautifulSoup                           #For scraping data
import time                                             #For waiting
import numpy as np                                      #For array
from tqdm import tqdm                                   #For progress bar

# OPENING REQUIRED SITE IN BROWSER

In [2]:
driver = webdriver.Firefox()

In [3]:
driver.get('https://m.imdb.com/chart/starmeter/')

# GETTING HTML CODE OF OPENED WEBPAGE

In [4]:
html = driver.page_source

# CONVERTING EXTRACTED HTML CODE INTO SOUP TO SCRAP DATA

In [5]:
soup = BeautifulSoup(html,"html.parser")

# EXTRACTING NAMES OF CELEBRITIES AND LINKS TO THEIR INDIVIDUAL WEBPAGES

In [6]:
a_list = soup.find_all("a")
name_list = []
link_list = []
for i in range(43,343,3):
    name_list.append(a_list[i].get('aria-label'))
    link_list.append(a_list[i].get('href'))

# EXTRACTING THE FAMOUS WORK OF THESE CELEBRITIES

In [7]:
famous_list = []
a_list1 = soup.findAll('a',{'data-testid':'nlib-known-for-title'})
for i in a_list1:
    famous_list.append(i.text)

# FUNCTION TO EXTRACT DATE OF BIRTH OF THE CELEBRITIES

In [8]:
def get_dob(soup):
    try:
        sec = soup.findAll('section', {'data-testid':'PersonalDetails'})[0]
        dob = sec.findAll('li',{'data-testid':'nm_pd_bl'})[0].findAll('li')[0].text
    except Exception as e:
        dob = np.nan
    return dob

# FUNCTION TO EXTRACT HEIGHT OF THE CELEBRITIES

In [9]:
def get_height(soup):
    try:
        sec = soup.findAll('section', {'data-testid':'PersonalDetails'})[0]
        height = sec.findAll('li',{'data-testid':'nm_pd_he'})[0].findAll('li')[0].text
    except Exception as e:
        height = np.nan
    return height

# FUNCTION TO EXTRACT BIRTHPLACE OF THE CELEBRITIES

In [10]:
def get_birthplace(soup):
    try:
        sec = soup.findAll('section', {'data-testid':'PersonalDetails'})[0]
        bp = sec.findAll('li',{'data-testid':'nm_pd_bl'})[0].findAll('li')[1].text
    except Exception as e:
        bp = np.nan
    return bp

# FUNCTION TO EXTRACT AWARD DATA OF THE CELEBRITIES

In [11]:
def get_awards(soup):
    try:
        div_list = soup.findAll('div', {'data-testid':'awards'})
        awards = div_list[0].findAll('li',{'class':'ipc-inline-list__item'})[0].text
    except Exception as e:
        awards = np.nan
    return awards

# FUNCTION TO EXTRACT ROLES OF THE CELEBRITIES

In [12]:
def get_role(soup):
    role_list = []
    try:
        div_list = soup.findAll('div', {'class':'sc-7c4234bd-0 jwsqEY'})[0].findAll('li')
        for i in div_list:
            role_list.append(i.text)
    except Exception as e:
        role_list.append(np.nan)
    return role_list

# MAIN LOOP TO ITERATE THROUGH EACH CELEBRITY'S WEBPAGE AND EXTRACT DATA

In [13]:
dob_list = []
height_list = []
role_list = []
award_list = []
birth_list = []
for i in tqdm(link_list, desc="Processing Links"):
    driver.get(f'https://m.imdb.com/{i}')
    #time.sleep(2)
    html = driver.page_source
    soup = BeautifulSoup(html,"html.parser")
    dob_list.append(get_dob(soup))
    height_list.append(get_height(soup))
    birth_list.append(get_birthplace(soup))
    role_list.append(get_role(soup))
    award_list.append(get_awards(soup))

Processing Links:   0%|          | 0/100 [00:00<?, ?it/s]

Processing Links: 100%|██████████| 100/100 [15:09<00:00,  9.10s/it]


# CREATING DATAFRAME FOR COLLECTED DATASET 

In [14]:
df = pd.DataFrame({
    'Rank':np.arange(1,101),
    'Name': name_list,
    'Date_of_birth': dob_list,
    'height': height_list,
    'Role': role_list,
    'Awards': award_list,
    'Famous_for': famous_list,
    'Birth_place': birth_list
})

# CHECKING THE DATASET

In [15]:
df.head()

,Rank,Name,Date_of_birth,height,Role,Awards,Famous_for,Birth_place
0,1,Matthew Perry,"August 19, 1969",6′ (1.83 m),"[Actor, Producer, Writer]",6 wins & 34 nominations total,Friends (1994–2004),"Williamstown, Massachusetts, USA"
1,2,Carla Gugino,"August 29, 1971",5′ 5″ (1.65 m),"[Actress, Producer, Soundtrack]",3 wins & 15 nominations,Watchmen (2009),"Sarasota, Florida, USA"
2,3,John Bennett Perry,"January 4, 1941",6′ 2″ (1.88 m),"[Actor, Soundtrack]",1 win & 1 nomination,George of the Jungle (1997),"Williamstown, Massachusetts, USA"
3,4,Elizabeth Lail,"March 25, 1992",5′ 8″ (1.73 m),[Actress],1 nomination,Five Nights at Freddy's (2023),"Williamson County, Texas, USA"
4,5,Shira Haas,"May 11, 1995",NaN,"[Actress, Soundtrack]",7 wins & 15 nominations total,Shtisel (2013–2021),"Tel Aviv, Israel"


In [16]:
df.tail()

,Rank,Name,Date_of_birth,height,Role,Awards,Famous_for,Birth_place
95,96,Robert De Niro,"August 17, 1943",5′ 9½″ (1.77 m),"[Actor, Producer, Director]",67 wins & 142 nominations total,Cape Fear (1991),"New York City, New York, USA"
96,97,Mckenna Grace,"June 25, 2006",5′ 0¾″ (1.54 m),"[Actress, Producer, Additional Crew]",3 wins & 12 nominations total,Gifted (2017),"Grapevine, Texas, USA"
97,98,Henry Cavill,"May 5, 1983",6′ 1″ (1.85 m),"[Actor, Additional Crew, Producer]",3 wins & 15 nominations,Man of Steel (2013),"Jersey, Channel Islands"
98,99,Brad Pitt,"December 18, 1963",5′ 11″ (1.80 m),"[Producer, Actor, Executive]",120 wins & 224 nominations total,Fight Club (1999),"Shawnee, Oklahoma, USA"
99,100,Carrie Coon,"January 24, 1981",5′ 5″ (1.65 m),"[Actress, Producer, Visual Effects]",6 wins & 47 nominations total,Gone Girl (2014),"Copley, Ohio, USA"


In [17]:
df.sample(20)

,Rank,Name,Date_of_birth,height,Role,Awards,Famous_for,Birth_place
0,1,Matthew Perry,"August 19, 1969",6′ (1.83 m),"[Actor, Producer, Writer]",6 wins & 34 nominations total,Friends (1994–2004),"Williamstown, Massachusetts, USA"
87,88,Anya Taylor-Joy,"April 16, 1996",5′ 8″ (1.73 m),"[Actress, Soundtrack]",19 wins & 44 nominations total,The Queen's Gambit (2020– ),"Miami, Florida, USA"
47,48,Jacob Fortune-Lloyd,"January 18, 1988",5′ 9″ (1.75 m),[Actor],NaN,Star Wars: Episode IX - The Rise of Skywalker ...,"Hillingdon, London, England, UK"
52,53,Sarah Carter,"October 30, 1980",5′ 6″ (1.68 m),"[Actress, Producer, Director]",8 wins & 6 nominations,Rogue (2015–2016),"Toronto, Ontario, Canada"
13,14,Lily Gladstone,"August 2, 1986",5′ 7″ (1.70 m),"[Actress, Casting Department, Writer]",8 wins & 21 nominations,Certain Women (2016),"Browning, Montana"
21,22,Rachael Harris,"January 12, 1968",5′ 1″ (1.55 m),"[Actress, Camera and Electrical Department, Pr...",2 wins & 2 nominations,The Hangover (2009),"Worthington, Ohio, USA"
60,61,Maddie Phillips,September 6,NaN,[Actress],NaN,Teenage Bounty Hunters (2020– ),NaN
22,23,Thora Birch,"March 11, 1982",5′ 4″ (1.63 m),"[Actress, Producer, Director]",17 wins & 27 nominations total,Ghost World (2001),"Los Angeles, California, USA"
56,57,Catherine O'Hara,"March 4, 1954",5′ 4″ (1.63 m),"[Actress, Writer, Producer]",33 wins & 42 nominations total,A Mighty Wind (2003),"Toronto, Ontario, Canada"
6,7,Josh Hutcherson,"October 12, 1992",5′ 5″ (1.65 m),"[Actor, Producer, Director]",19 wins & 25 nominations,The Hunger Games (2012),"Union, Kentucky, USA"


# SAVING THE DATASET

In [18]:
df.to_csv('celebrity_data.csv',index= False)